### **SLAM** (Simultaneous Localization and Mapping)

SLAM is a technique used in robotics and computer vision to create a **`map`** of an unknown environment while simultaneously keeping track of the **`agent's location`** within that environment. It combines data from various sensors, such as cameras and LiDAR, to build a 2D (grid maps) or 3D map (3D point cloud) and localize the robot within it.

Two main techniques used in SLAM are:
1. Build a map of an unknown environment (mapping).
2. Track its own pose (position and orientation) within that map at the same time (localization).

Core functionalities of Slam Algorithms:

1. `Sensor inputs`
- SLAM typically uses sensor data (e.g., LIDAR scans, camera images, or depth sensor measurements) to detect features or landmarks in the environment.
2. `State estimation`
- An internal state (the robot’s pose, including x, y, yaw) is estimated using algorithms like Extended Kalman Filters, Particle Filters, or Graph Optimization.
3. `Map building`
- As the robot moves, it accumulates new sensor data. The SLAM algorithm integrates that data into a global map (2D grid map, 3D point cloud, or other representations).
4. `Loop closure`
- When the robot revisits a previously mapped area, the SLAM algorithm detects that it’s the same place (loop closure). This knowledge is used to reduce accumulated drift and refine both the map and pose estimates.





Slam toolbox:

In [ ]:
sudo apt install ros-jazzy-slam-toolbox

There is a new node in the launch file marker_server from the `interactive-marker-twist-server` this can be used to `move and rotate our robot directly from **RViz***`. Let's install it with:

In [ ]:
sudo apt install ros-jazzy-interactive-marker-twist-server

We moved all the `parameter_bridge` topics into a yaml config file `gz_bridge.yaml`, we don't describe them in the launch file anymore, this is more comfortable when we forward many topics

`base_link` → robot’s body center.

`odom` frame → comes from wheel encoders (`local` odometry frame).
- It estimates how much your `robot moved` — but it `drifts` over time (small errors add up).

`map` frame → comes from SLAM (e.g. from LiDAR-based mapping, `Global` reference frame).
- It constantly `corrects` that drift by `matching sensor data with the map`.


So, if your odometry drifts, the SLAM algorithm shifts the robot’s map pose to keep it consistent with real-world features.
The difference between map and odom = how much drift your raw odometry accumulated since SLAM had to correct it.



---
#### **MAPPING**


SLAM tool box offers 2 ways to store maps:

1. Save Map  
- Command: `ros2 run nav2_map_server map_saver_cli -f my_map`  
- Output:  
  - `my_map.pgm` → grayscale image (white = free, black = occupied)  
  - `my_map.yaml` → metadata (resolution, origin, etc.)  
- Use: for localization only later (e.g. `nav2_amcl`)  
- Limitation: cannot resume mapping since it's just an image  
- Analogy: like taking a screenshot of your map  

2. Serialize Map  
- Command: `ros2 service call /slam_toolbox/serialize_map ...`  
- Saves SLAM Toolbox’s internal graph (poses, scans, loop closures, etc.)  
- Allows later deserialization to continue mapping from the same state  
- Use: for continuing SLAM sessions, not for localization with other nodes  
- Analogy: like saving a project file to edit later  

**Note:** In Gui windows type the name of both save and serialize files, and click once , they will be automatically saved in the *_ws folder, we must be in the *_ws folder during launch file.

3. Deserialize Map  
- Command: `ros2 service call /slam_toolbox/deserialize_map ...`
- Loads a previously serialized map to resume SLAM
- Use: to continue mapping from where you left off
- Limitation: cannot use deserialized maps for localization with other nodes
- Analogy: like opening a saved project file to continue working

**Note:** Type name of any saved serialize_map without any extension (both .data and .posegraph files should be in the *_ws folder) in the Gui window and click once, it will be automatically loaded.



---

#### **Custom deserialize map executable Working**

1. MapLoaderNode which calls the service to deserialize:

- Doesn’t open or parse the map itself. Calls a ROS2 service:
`/slam_toolbox/deserialize_map`
(service type slam_toolbox/srv/DeserializePoseGraph).

- That service is implemented inside the slam_toolbox node, not by us. So, when our node sends this request:
`self.request.filename = "/root/nav_ws/install/ros2_nav/share/ros2_nav/maps/serialized"`

- the slam_toolbox node receives it, opens that serialized file (a .posegraph file), and reconstructs:
   - its internal pose graph,
   - the map data,
   - and all the robot poses stored inside it.

- Essentially, slam_toolbox “loads” the map into its running memory, not into another file.

2. Once slam_toolbox finishes deserializing:

- It populates its internal occupancy grid, constraints, and graph nodes.
- Then `it starts advertising topics` like:
   - /map (OccupancyGrid)
   - /map_metadata
   - /slam_toolbox/pose_graph_visualization
   - /slam_toolbox/map_updates
- It also provides the /map -> /odom transform via TF2.

That’s the `shared state` that `other launch files` (like navigation or localization launch) use. they don’t reload the serialized file themselves.
Other `launch files/nodes` just `connect to those ROS2 topics/services`.

---

#### **LOCALIZATION**

While mapping was the process of creating a representation (a map) of an environment. Localization is the process by which a `robot determines its own position and orientation within a known environment (map)`. In other words:

- The environment or map is typically already available or pre-built.
- The robot’s task is to figure out “Where am I?” or “Which direction am I facing?” using sensor data, often by `matching its current perceptions to the known map`.


#### Localization with AMCL
`AMCL` (Adaptive Monte Carlo Localization) is a `particle filter–based 2D localization algorithm`. The robot’s possible `poses` (position + orientation in 2D) are represented by a `set of particles`. It adaptively samples the robot’s possible poses according to sensor readings and motion updates, converging on an accurate estimate of where the robot is within a known map.

AMCL is part of the ROS2 navigation stack, let's install it first:

In [ ]:
sudo apt install ros-jazzy-nav2-bringup 
sudo apt install ros-jazzy-nav2-amcl

Write a separate launch file for localization, name it `localization.launch.py` in the `launch` folder of your package. include rviz_config_arg, rviz_launch_arg, sim_time_arg, nav2_localization_launch_path (Nav2 bringup),  localization_params_path(amcl_localization.yaml), map_file_path (my_map.yaml), rviz_node, sim_time_arg,    interactive_marker_twist_server_node, localization_launch.


[amcl_localization.yaml](https://docs.nav2.org/configuration/packages/configuring-amcl.html) (Link from official doc of Nav2). We have to set this for our localization.launch.py file.

[Nav2 Documentation](https://docs.nav2.org/index.html)

The `main purpose` of the `localization algorithm` is establishing the `transformation` between the `fixed map` and the `robot's odometry frame` based on `real time sensor data`. We can visualize this in RViz as we saw it during mapping.

---


#### Correlation vs Covariance 
- Correlation describe the direction and strength of relationship between two Random variables.
- Covariance indicates the extent/direction to which two random variables change together.

<img src="assets/c1.png" width="800" height="600"/>


**`Population`**
All possible data points in a group you want to study.
Example: the heights of all students in a university.

**`Sample`**
A smaller subset taken from the population to estimate its characteristics.
Example: heights of 50 students selected from that university.

<img src="assets/c2.png" width="750" height="600"/>




<img src="assets/c3.png" width="375" height="200"/>
<img src="assets/c4.png" width="375" height="200"/>

<img src="assets/c5.png" width="750" height="600"/>

---

<img src="assets/c6.png" width="750" height="300"/>
<br>
<img src="assets/c7.png" width="750" height="500"/>

<img src="assets/c8.png" width="600" height="200"/>
<br>
<img src="assets/c9.png" width="600" height="550"/>
<br>

**`Variance`**
- Measures how far each data point deviates from the mean on average (spread of data).
- Example: if exam scores are 70, 80, 90, the variance shows how far each score lies from the mean 80.

**`Standard deviation`**
- It is the square root of variance, giving spread in the same unit as the data.
- Example: if variance of exam scores is 25, then standard deviation is 5, meaning scores typically differ by 5 marks from the mean.

<img src="assets/c10.png" width="600" height="450"/>

---


#### Covariance Matrix:

<img src="assets/c11.png" width="750" height="600"/>
<br>
<img src="assets/c13.png" width="750" height="600"/>


<img src="assets/c14.png" width="750" height="300"/>

---


#### **Pose Estimation**
- Initial Pose Calculation of Robot using Covariance Matrix


AMCL needs the `initial pose` which can be given using `RviZ's` `2D Estimate Pose` tool. Then AMCL's particles start appearing on the RviZ, each particle represents a pose [x,y,yaw(0)]. 

We can define our own Custom node for `initial pose sending` and run using `ros2 run ros2_nav_py send_initial_pose` which will be a publisher node and publish at `/initialpose` which is required by AMCL

<img src="assets/c15.png" width="750" height="550"/>
<img src="assets/c16.png" width="750" height="550"/>
<img src="assets/c17.png" width="750" height="600"/>
<img src="assets/c18.png" width="750" height="600"/>

----

#### **Eulear Angles (rpy) vs Quaternions (qx,qy,qz,qw)**



<img src="assets/q2.png" width="750" height="500"/>

<img src="assets/q6.png" width="750" height="500"/>

<img src="assets/q3.png" width="750" height="600"/>

<img src="assets/q1.png" width="750" height="600"/>
<img src="assets/q4.png" width="750" height="500"/>
<img src="assets/quaternions.png" width="750" height="500"/>
<img src="assets/q5.png" width="750" height="600"/>

- `Euler angles` use `sequential rotations` and suffer from `singularities` at specific configurations.
- `Quaternions` use a `unified 4D representation` based on axis–angle formulation, preserving smoothness and numerical stability across all possible orientations.

Links: (also in assets folder)

[Eulear Angles](https://danceswithcode.net/engineeringnotes/rotations_in_3d/rotations_in_3d_part1.html)

[Quaternions](https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html)

---

#### **Localization with SLAM**

It's possible to use SLAM toolbox in `localization mode`, it requires a small adjustment on the parameters which is already part of this lesson, slam_toolbox_localization.yaml. This requires the path to the serialized map file.Make sure the map_file_name parameter is changed to the path on your machine to the serialized map file! Create the launch file for localization, localization_slam_toolbox.launch.py, it's very similar to the SLAM toolbox mapping launch file.

Rebuild the workspace and try it, we'll see that the map keeps updating unlike with AMCL, this is the normal behavior of the localization with SLAM toolbox. According to this [paper](https://joss.theoj.org/papers/10.21105/joss.02783), the localization mode does continue to update the pose-graph with new constraints and nodes, but the updated map expires over some time. SLAM toolbox describes it as "elastic", which means it holds the updated graph for some amount of time, but does not add it to the permanent graph.

----

### **Navigation**
Navigation in robotics is the overall process that enables a robot to move from one location to another in a safe, efficient, and autonomous manner. It typically involves:

- Knowing where the robot is (`localization or SLAM`),
- Knowing where it needs to go (a `goal pose or waypoint`),
- Planning a path to reach that goal (`path planning`), and
- Moving along that path while avoiding dynamic and static obstacles (`motion control` and `obstacle avoidance`).

ROS's nav2 navigation stack implements the above points 2 to 4, for the first point we already met several possible solutions.

---


#### **Navigation with SLAM**

As we saw to navigate a mobile robot we need to know 2 things:

- Knowing where the robot is (localization or SLAM),
- Knowing where it needs to go (a goal pose or waypoint)
In the previous examples we used `localization on a known map`, but it's also possible to navigate together with an `online SLAM`. It means we don't know the complete environment around the robot but we can already navigate in the known surrounding.

Create a navigation_with_slam.launch.py launch file where we start both the `SLAM toolbox` and the `navigation stack`.

---

#### **Exploration**
Exploration is a process by which a robot operating in an `unknown` or partially known `environment` actively `searches` the space to gather new information. This is a real life use case to use SLAM together with the navigation stack.


git clone https://github.com/MOGI-ROS/m-explore-ros2.git
Build the workspace and source the setup.bash to make sure ROS is aware about the new package!

We'll need 3 terminals, one for the simulation:

ros2 launch bme_ros2_navigation spawn_robot.launch.py
In another terminal we launch the navigation_with_slam.launch.py:

ros2 launch bme_ros2_navigation navigation_with_slam.launch.py
And in the third one we launch the exploration:

ros2 launch explore_lite explore.launch.py
The exploration node will identify the boundaries of the know surrounding and will navigate the robot until it finds all the physical boundaries of the environment.